In [ ]:
import ee
import numpy as np
from PIL import Image
import cv2
import os
import time
from pathlib import Path

In [ ]:
def geeInit():
    # Authentication of google earth engine to access some files as example the local relief model
    ee.Authenticate()
    # Initialization of google earth engine
    ee.Initialize()

    #this function gets the dimention in pixel of LRM image
def getDimentionsImage(img):
  imgDescription = ee.Algorithms.Describe(img)
  w = ee.List(ee.Dictionary(ee.List(ee.Dictionary(imgDescription).get("bands")).get(0)).get("dimensions")).get(0)
  w = w.getInfo()
  h = ee.List(ee.Dictionary(ee.List(ee.Dictionary(imgDescription).get("bands")).get(0)).get("dimensions")).get(1)
  h = h.getInfo()

  return w, h

#this function uses the equations before obtained to convert a geo-coordinate to pixel coordinate
def convert_GeoCoord_to_PixelCoord(geocoordinate, m1, m2, b1, b2):
  x_geo = geocoordinate[0]
  y_geo = geocoordinate[1]
  x = round(x_geo * m1 + b1)
  y = round(y_geo * m2 + b2)
  
  return x, y

#this function is the inverse of the previous function, with it we can get the geo-coordinates that corresponds to the pixel coordinates
def convert_PixelCoord_to_GeoCoord(pixelcoordinate, m1, m2, b1, b2):
  x = pixelcoordinate[0]
  y = pixelcoordinate[1]
  x_geo = (x - b1)/m1
  y_geo = (y - b2)/m2
  
  return x_geo, y_geo

In [ ]:
geeInit()

In [ ]:
LRM = ee.Image('users/fabriziobotelho/tiff_files')
geometry = LRM.geometry()
lrm_info = LRM.getInfo()
print(lrm_info) #debug

#geo-coordinates of upper left and lower right corners of LRM image
upper_left = [-8.907277129492169,42.16335245423417]
lower_right = [-8.06051504207768,41.59280385023923]

points = ee.Geometry.MultiPoint([upper_left, lower_right])

In [ ]:
width, height = getDimentionsImage(LRM)
width, height

In [ ]:
x_geo1 = upper_left[0]
y_geo1 = upper_left[1]
x_geo2 = lower_right[0]
y_geo2 = lower_right[1]

x1 = 0;
y1 = 0;
x2 = width - 1 
y2 = height - 1

#equations to convert geo-coordinates to pixel coordinates
#x = x_geo * m1 + b1
#y = y_geo * m2 + b2

m1 = (x1 - x2)/(x_geo1 - x_geo2)
b1 = x1 - x_geo1 * m1
print('m1:', m1, 'b1:', b1)

m2 = (y1 - y2)/(y_geo1 - y_geo2)
b2 = y1 - y_geo1 * m2
print('m2:', m2, 'b2:', b2)

In [ ]:
resolution = 250

numberColumns = round((width) / resolution)
numberRows = round((height)/ resolution)
print('numberColumns, numberRows:', numberColumns, numberRows)
numberColumns*numberRows

In [ ]:
#In this part of code is constructed a block that will go through the LRM image and crop/export the image inside that block 
#if the number of archaelogical objects is greater than zero
#so the images cropped has the 320x320 of dimension and the scale used was 1 meter by pixel, that means each image corresponds to 320 x 320 meters on terrain

start_time = time.time()

#initialize parameters
x11 = 0
x12 = 0
x21 = resolution
x22 = 0
x31 = resolution
x32 = resolution
x41 = 0
x42 = resolution

i = 0
j = 0

while i < numberRows:
    while j < numberColumns:
        x1 = convert_PixelCoord_to_GeoCoord([x11, x12], m1, m2, b1, b2)
        x2 = convert_PixelCoord_to_GeoCoord([x21, x22], m1, m2, b1, b2)
        x3 = convert_PixelCoord_to_GeoCoord([x31, x32], m1, m2, b1, b2)
        x4 = convert_PixelCoord_to_GeoCoord([x41, x42], m1, m2, b1, b2)
        
        block = ee.Geometry.Polygon([[x1[0], x1[1]], [x2[0], x2[1]], [x3[0], x3[1]], [x4[0], x4[1]]])

        img = LRM.clipToBoundsAndScale(block, None, None, None, 0.5)
        img = img.unmask()

        band_arrs = img.sampleRectangle(block)
        band_arrs = band_arrs.get('b1')

        # Transfer the arrays from server to client and cast as np array.
        np_arr_b1 = np.array(band_arrs.getInfo())  
        array = np.array(np_arr_b1, dtype='uint8')
        im = Image.fromarray(array)
        im.save("C:\\Users\\fabri\\Downloads\\blocks\\"+str(i)+"_"+str(j)+".jpg")

        x11 = x11 + resolution
        x12 = x12
        x21 = x21 + resolution
        x22 = x22
        x41 = x41 + resolution
        x42 = x42 
        x31 = x31 + resolution
        x32 = x32
        
        j = j + 1
    x11 = 0
    x21 = resolution
    x41 = 0
    x31 = resolution

    x12 = x12 + resolution
    x22 = x22 + resolution
    x42 = x42 + resolution
    x32 = x32 + resolution
    j = 0
    i = i + 1
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# concatenate horizontal images
directory = "C:\\Users\\fabri\\Downloads\\blocks\\"

scale = 0.5
dim = (500, 500)

# read image
paths = sorted(Path(directory).iterdir(), key=os.path.getmtime)
imgs = []
for k in range(len(paths)):
    imgs.append(str(paths[k]))

i = 0
for row in range(numberRows):
    imageA = cv2.imread(imgs[i], cv2.IMREAD_GRAYSCALE)
    imageA = cv2.resize(imageA, dim, interpolation = cv2.INTER_AREA)
    for col in range(i+1, numberColumns+i):
        imageB = cv2.imread(imgs[col], cv2.IMREAD_GRAYSCALE)
        imageB = cv2.resize(imageB, dim, interpolation = cv2.INTER_AREA)
        imageA = np.hstack((imageA, imageB))
    i = col + 1
    cv2.imwrite("D:\\row\\"+str(row)+".tif", imageA)

    

In [ ]:
# concatenate vertical images 
# read image
directory = "D:\\row"
paths = sorted(Path(directory).iterdir(), key=os.path.getmtime)
imgs = []
for k in range(len(paths)):
    imgs.append(str(paths[k]))

imageA = cv2.imread(imgs[0], cv2.IMREAD_GRAYSCALE)
for row in range(numberRows):
    imageB = cv2.imread(imgs[row], cv2.IMREAD_GRAYSCALE)
    imageA = np.vstack((imageA, imageB))
    print("Row:", row, "finished.")
print("Writing image...")
cv2.imwrite("E:\\LRM_image\\LRM_image.tif", imageA)    
print("Finished.")